In [7]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# Load data
titles = pd.read_csv('./data_unzipped/netflix_titles.csv')
subscribers = pd.read_csv('./netflix_sub_count.csv')
subscribers.columns = ['quarter', 'sub_millions']
# Change date_added to datetime
titles['date_added'] = pd.to_datetime(titles['date_added'])
# Add quarter column
titles['quarter'] = titles['date_added'].dt.to_period('Q').astype(str)
# Fix rows where duration was input to rating
titles.loc[5541, 'duration'] = titles.loc[5541, 'rating']
titles.loc[5794, 'duration'] = titles.loc[5794, 'rating']
titles.loc[5813, 'duration'] = titles.loc[5813, 'rating']
# Fix NA ratings by googling the title
titles.loc[5541, 'rating'] = 'TV-MA'
titles.loc[5794, 'rating'] = 'TV-MA'
titles.loc[5813, 'rating'] = 'TV-MA'
titles.loc[7537, 'rating'] = 'PG-13'
titles.loc[5989, 'rating'] = 'TV-PG'
titles.loc[7312, 'rating'] = 'TV-G'
titles.loc[6827, 'rating'] = 'TV-14'
# Create a new column to better house duration
titles['length'] = titles['duration'].str.extract('(\d+)').astype(int)
# Random data fixes
titles.loc[4653, 'country'] = 'United States'

In [9]:
# Create a dataframe of directors and their counts
directors = titles['director'].str.split(', ').explode().value_counts().reset_index()
directors.columns = ['value', 'count']
# Create a dataframe of actors and their counts
actors = titles['cast'].str.split(', ').explode().value_counts().reset_index()
actors.columns = ['value', 'count']
# Create a dataframe of countries and their counts
countries = titles['country'].str.split(', ').explode().value_counts().reset_index()
countries.columns = ['value', 'count']
# Create a dataframe of genres and their counts
genres = titles['listed_in'].str.split(', ').explode().value_counts().reset_index()
genres.columns = ['value', 'count']

In [10]:
# Create a function to create a new column based on the top quantile of a column
def top_quantile_criteria(element, counts, threshold):
    if pd.isna(element):
        return 0
    for i in element.split(', '):
        if counts[counts['value'] == i]['count'].values[0] >= threshold:
            return 1
    return 0
# Create a list of thresholds for directors and actors
thresholds = [
    (directors['count'].quantile(0.9), actors['count'].quantile(0.9)),
    (directors['count'].quantile(0.95), actors['count'].quantile(0.95)),
    (directors['count'].quantile(0.99), actors['count'].quantile(0.99))
]
# Create columns for directors and actors in the top 10%, 5%, and 1% of counts
for dir_t, act_t in thresholds:
    titles[f'dir_in_{int(dir_t)}'] = titles['director'].apply(lambda x: top_quantile_criteria(x, directors, dir_t))
    titles[f'act_in_{int(act_t)}'] = titles['cast'].apply(lambda x: top_quantile_criteria(x, actors, act_t))

In [11]:
def check_in(element, interest):
    if pd.isna(element):
        return 0
    for i in element.split(', '):
        if i == interest:
            return 1
    return 0
# Add a new binary column for titles from top 10 countries and top 20 genres and all ratings
top_10_countries = countries['value'].head(10).tolist()
top_20_genres = genres['value'].head(20).tolist()
ratings = titles['rating'].unique().tolist()
for i in top_10_countries:
    titles[f'{i}'] = titles['country'].apply(lambda x: check_in(x, i))
for i in top_20_genres:
    titles[i] = titles['listed_in'].apply(lambda x: 1 if i in x else 0)
for i in ratings:
    titles[i] = titles['rating'].apply(lambda x: 1 if i in x else 0)
# Binary column for tv show and movie
titles['tv_show'] = titles['type'].apply(lambda x: 1 if x == 'TV Show' else 0)
titles['movie'] = titles['type'].apply(lambda x: 1 if x == 'Movie' else 0)

In [12]:
# make a new df with all of the binary columns and quarter column
df = titles[
    ['quarter', 'release_year', 'length',
     'tv_show', 'movie', 'dir_in_2',
     'dir_in_3', 'dir_in_6', 'act_in_3',
     'act_in_5', 'act_in_10'] + top_10_countries + top_20_genres + ratings]
# slice df into tv shows and movies. summarize both by quarter to get avg length
df_tv = df[df['tv_show'] == 1][['quarter', 'length']].groupby('quarter').mean().reset_index()
df_movie = df[df['movie'] == 1][['quarter', 'length']].groupby('quarter').mean().reset_index()
# merge the two dfs and rename columns
length = df_tv.merge(df_movie, on='quarter')
length.columns = ['quarter', 'avg_tv_length', 'avg_movie_length']
# summarise df by quarter and combine with subscriber data
subscribers = subscribers.merge(length, on='quarter')
df = subscribers.merge(df.groupby('quarter').sum().reset_index(), on='quarter')
# Add a column for total titles added
df['total'] = df['tv_show'] + df['movie']

In [13]:
# save as subs.csv
df.to_csv('./subs.csv', index=False)